In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
df_cleaned=pd.read_csv('/content/selected_sba_loans_data (2).csv')

In [ ]:
df_cleaned.dropna(inplace=True)
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613723 entries, 0 to 613722
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ChgOffPrinGr       613723 non-null  float64
 1   Term               613723 non-null  float64
 2   DisbursementGross  613723 non-null  float64
 3   UrbanRural         613723 non-null  int64  
 4   RevLineCr          613723 non-null  int64  
 5   LowDoc             613723 non-null  int64  
 6   BankState          613723 non-null  int64  
 7   State              613723 non-null  int64  
 8   NAICS              613723 non-null  int64  
 9   MIS_Status         613723 non-null  int64  
dtypes: float64(3), int64(7)
memory usage: 46.8 MB


In [ ]:
# Define your feature set and target
X = df_cleaned[['ChgOffPrinGr', 'Term', 'UrbanRural', 'RevLineCr', 'LowDoc', 'BankState', 'State', 'NAICS']]  # Exclude DisbursementGross
y = df_cleaned['MIS_Status']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Store DisbursementGross separately for later use
disbursement_test = df_cleaned.loc[X_test.index, 'DisbursementGross']  # Align DisbursementGross with X_test

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Initialize KNN
knn = KNeighborsClassifier()

# Define parameter grid
param_grid = {
    'n_neighbors': range(1, 31),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': [1, 2]
}

# GridSearchCV
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'metric': 'manhattan', 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Best Score: 0.917996325404373


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define parameter distributions
param_distributions = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': np.arange(2, 20, 2),
    'min_samples_leaf': np.arange(1, 10, 1),
    'max_features': [None, 'sqrt', 'log2']
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dt,  # Correctly initialized estimator
    param_distributions=param_distributions,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    random_state=42,
    verbose=1
)

# Fit the model
random_search.fit(X_train, y_train)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=50,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 5, 10, 20],
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])},
                   random_state=42, scoring='accuracy', verbose=1)

In [ ]:
# Best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'min_samples_split': 18, 'min_samples_leaf': 7, 'max_features': None, 'max_depth': 5, 'criterion': 'gini'}
Best Score: 0.993722733176518


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np


# Initialize BaggingClassifier with a base estimator
bagging = BaggingClassifier(random_state=42)

# Define hyperparameter distributions
param_distributions = {
    'estimator': [DecisionTreeClassifier(max_depth=3), DecisionTreeClassifier(max_depth=5),DecisionTreeClassifier(max_depth=10)],
    'n_estimators': [10, 50, 100, 200],        # Number of base estimators
    'max_samples': [0.5, 0.7, 1.0],            # Fraction of samples for each base estimator
    'max_features': [0.5, 0.7, 1.0],           # Fraction of features for each base estimator
    'bootstrap': [True, False],                # With or without replacement for samples
    'bootstrap_features': [True, False]        # With or without replacement for features
}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=bagging,
    param_distributions=param_distributions,
    n_iter=20,             # Number of random combinations to try
    scoring='accuracy',     # Evaluation metric
    cv=5,                   # 5-fold cross-validation
    verbose=1,              # Print progress
    random_state=42,        # Ensure reproducibility
    n_jobs=-1               # Use all CPU cores
)

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validated Score:", random_search.best_score_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'n_estimators': 10, 'max_samples': 1.0, 'max_features': 1.0, 'estimator': DecisionTreeClassifier(max_depth=5), 'bootstrap_features': False, 'bootstrap': False}
Best Cross-Validated Score: 0.9937044023457577


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report

# Initialize and fit bagging classifier
bagging = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42)
bagging.fit(X_train, y_train)

# Predict and evaluate
y_pred_bagging = bagging.predict(X_test)
print("Bagging Classification Report:\n", classification_report(y_test, y_pred_bagging))

Bagging Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00    100650
           1       0.97      0.99      0.98     22095

    accuracy                           0.99    122745
   macro avg       0.98      0.99      0.99    122745
weighted avg       0.99      0.99      0.99    122745



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Initialize Gradient Boosting
gb = GradientBoostingClassifier(random_state=42)

# Define hyperparameter distributions
param_distributions = {
    'learning_rate': np.linspace(0.01, 0.2, 20),   # Shrinkage
    'n_estimators': [100, 200, 300, 400, 500],     # Number of boosting stages
    'max_depth': [3, 4, 5, 6, 7],                  # Tree depth
    'min_samples_split': [2, 5, 10],               # Minimum samples to split
    'min_samples_leaf': [1, 2, 4],                 # Minimum samples at leaf
    'max_features': ['sqrt', 'log2', None],        # Number of features for splits
    'subsample': [0.5, 0.7, 1.0]                   # Fraction of samples per tree
}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb,
    param_distributions=param_distributions,
    n_iter=50,             # Number of random combinations
    scoring='accuracy',     # Evaluation metric
    cv=5,                   # 5-fold cross-validation
    random_state=42,        # Ensure reproducibility
    verbose=1,
    n_jobs=-1               # Use all CPU cores
)

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validated Score:", random_search.best_score_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'subsample': 1.0, 'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 4, 'learning_rate': 0.02}
Best Cross-Validated Score: 0.9937044023664996


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification

# Initialize XGBoost Classifier with parameters
xgb_model = xgb.XGBClassifier(
    n_estimators=100,        # Number of trees
    max_depth=4,             # Maximum tree depth
    learning_rate=0.02,      # Learning rate (similar to Gradient Boosting)
    min_child_weight=2,      # Equivalent to min_samples_leaf
    subsample=1.0,           # Use all samples
    colsample_bytree=1.0,    # Use all features
    objective='binary:logistic',  # Objective for binary classification
    random_state=42          # For reproducibility
)

# Perform cross-validation to evaluate performance
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='accuracy')

print("Cross-Validated Accuracy:", cv_scores.mean())

# Train the model on the training set
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate performance on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Set Accuracy:", test_accuracy)


Cross-Validated Accuracy: 0.9932196552472939
Test Set Accuracy: 0.9931565440547476


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification

# Initialize LightGBM Classifier with parameters
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,        # Number of trees
    max_depth=4,             # Maximum tree depth
    learning_rate=0.02,      # Learning rate
    min_child_samples=2,     # Minimum samples at leaf (similar to min_samples_leaf)
    subsample=1.0,           # Use all samples (subsample fraction)
    colsample_bytree=1.0,    # Use all features (feature fraction)
    random_state=42          # For reproducibility
)

# Perform cross-validation to evaluate performance
cv_scores = cross_val_score(lgb_model, X_train, y_train, cv=5, scoring='accuracy')

print("Cross-Validated Accuracy:", cv_scores.mean())

# Train the model on the training set
lgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lgb_model.predict(X_test)

# Evaluate performance on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Set Accuracy:", test_accuracy)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 71067, number of negative: 321715
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056835 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 877
[LightGBM] [Info] Number of data points in the train set: 392782, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.180932 -> initscore=-1.510043
[LightGBM] [Info] Start training from score -1.510043
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Initialize and fit Logistic Regression models with L1, L2, and ElasticNet penalties
lasso = LogisticRegressionCV(
    cv=5, penalty='l1', solver='saga', max_iter=1000, random_state=42
).fit(X_train, y_train)

ridge = LogisticRegressionCV(
    cv=5, penalty='l2', solver='lbfgs', max_iter=1000, random_state=42
).fit(X_train, y_train)

elastic = LogisticRegressionCV(
    cv=5, penalty='elasticnet', solver='saga', l1_ratios=[0.5], max_iter=1000, random_state=42
).fit(X_train, y_train)

# Evaluate and print classification reports
print("Lasso Classification Report:\n", classification_report(y_test, lasso.predict(X_test)))
print("Ridge Classification Report:\n", classification_report(y_test, ridge.predict(X_test)))
print("ElasticNet Classification Report:\n", classification_report(y_test, elastic.predict(X_test)))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which 

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Initialize and fit neural network with different activation functions
activations = ['relu', 'tanh', 'logistic']

for activation in activations:
    print(f"Training Neural Network with Activation Function: {activation}")
    nn = MLPClassifier(
        hidden_layer_sizes=(50, 25),
        activation=activation,
        max_iter=500,
        random_state=42
    )
    nn.fit(X_train, y_train)

    # Predict and evaluate
    y_pred_nn = nn.predict(X_test)
    print(f"Neural Network Classification Report for {activation}:\n", classification_report(y_test, y_pred_nn))
    print("-" * 60)


Training Neural Network with Activation Function: relu
Neural Network Classification Report for relu:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00    100650
           1       0.97      0.98      0.98     22095

    accuracy                           0.99    122745
   macro avg       0.99      0.99      0.99    122745
weighted avg       0.99      0.99      0.99    122745

------------------------------------------------------------
Training Neural Network with Activation Function: tanh
Neural Network Classification Report for tanh:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99    100650
           1       0.98      0.88      0.93     22095

    accuracy                           0.98    122745
   macro avg       0.98      0.94      0.96    122745
weighted avg       0.98      0.98      0.97    122745

------------------------------------------------------------
Training Neural Ne

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


# Initialize and fit neural network with different activation functions and solvers
activations = ['relu']
solvers = ['adam', 'lbfgs', 'sgd']

for activation in activations:
    for solver in solvers:
        print(f"Training Neural Network with Activation Function: {activation} and Solver: {solver}")
        nn = MLPClassifier(
            hidden_layer_sizes=(50, 25),
            solver=solver,
            activation=activation,
            max_iter=500,
            random_state=42
        )
        nn.fit(X_train, y_train)

        # Predict and evaluate
        y_pred_nn = nn.predict(X_test)
        print(f"Neural Network Classification Report for Activation: {activation}, Solver: {solver}\n", classification_report(y_test, y_pred_nn))
        print("-" * 60)


Training Neural Network with Activation Function: relu and Solver: adam
Neural Network Classification Report for Activation: relu, Solver: adam
               precision    recall  f1-score   support

           0       1.00      0.99      1.00    100650
           1       0.97      0.98      0.98     22095

    accuracy                           0.99    122745
   macro avg       0.99      0.99      0.99    122745
weighted avg       0.99      0.99      0.99    122745

------------------------------------------------------------
Training Neural Network with Activation Function: relu and Solver: lbfgs


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Neural Network Classification Report for Activation: relu, Solver: lbfgs
               precision    recall  f1-score   support

           0       0.96      0.99      0.98    100650
           1       0.96      0.82      0.89     22095

    accuracy                           0.96    122745
   macro avg       0.96      0.91      0.93    122745
weighted avg       0.96      0.96      0.96    122745

------------------------------------------------------------
Training Neural Network with Activation Function: relu and Solver: sgd
Neural Network Classification Report for Activation: relu, Solver: sgd
               precision    recall  f1-score   support

           0       0.82      1.00      0.90    100650
           1       0.00      0.00      0.00     22095

    accuracy                           0.82    122745
   macro avg       0.41      0.50      0.45    122745
weighted avg       0.67      0.82      0.74    122745

------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


 Rule of Thumb for Neurons in Hidden Layers:
Number of neurons in the first hidden layer =
Input Neurons
+
Output Neurons
2
2
Input Neurons+Output Neurons
​
 .
Example: If input features = 20 and output classes = 2, start with
20
+
2
2
=
11
2
20+2
​
 =11 neurons in the first layer.

In [ ]:
from sklearn.neural_network import MLPClassifier

# Define the neural network with 9 input features, 2 hidden layers, and 2 output features
model = MLPClassifier(hidden_layer_sizes=(16, 8), activation='relu', solver='adam', max_iter=500, random_state=42)

# Fit the model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99    100650
           1       0.97      0.98      0.98     22095

    accuracy                           0.99    122745
   macro avg       0.98      0.99      0.99    122745
weighted avg       0.99      0.99      0.99    122745



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np


# Define the parameter grid for RandomizedSearchCV
param_dist = {
    'hidden_layer_sizes': [(np.random.randint(5, 101), np.random.randint(5, 101)) for _ in range(10)],  # Try different 2-layer combinations
    'activation': ['relu'],
    'solver': ['adam'],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [500, 1000]
}

# Initialize the MLPClassifier
mlp = MLPClassifier(random_state=42)

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(mlp, param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)

# Fit the model
random_search.fit(X_train, y_train)

# Print the best parameters found by RandomizedSearchCV
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best Cross-Validated Score: {random_search.best_score_}")

# Evaluate the model with the test set
y_pred = random_search.best_estimator_.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))


Best Parameters: {'solver': 'adam', 'max_iter': 500, 'learning_rate': 'constant', 'hidden_layer_sizes': (51, 66), 'activation': 'relu'}
Best Cross-Validated Score: 0.9924843887878251
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00    100650
           1       0.97      0.98      0.98     22095

    accuracy                           0.99    122745
   macro avg       0.99      0.99      0.99    122745
weighted avg       0.99      0.99      0.99    122745



In [ ]:
###Hyperparameter choices
